In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd

import time
import os.path
import MapMatching4GMNS

MapMatching4GMNS, version 0.2.14


In [2]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

## step 1 Convert utd Data into GMNS Format

In [100]:
"Convert utd Data into GMNS Format"
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[:5] == 'links':
        utd = pd.read_csv(file)
    if file[:9] == 'detectors':
        detectors = pd.read_csv(file)
        break
utd.rename(columns = {'Unnamed: 0':'utd','long':'utd_long','lat':'utd_lat','citycode':'utd_citycode'}, inplace = True)
detectors.rename(columns = {'Unnamed: 0':'detectors','long':'detectors_long',\
                'lat':'detectors_lat','citycode':'detectors_citycode','length':'detectors_length'}, inplace = True)
detectors['linkid'] = detectors['linkid'].astype(int)
detectors['lanes'] = detectors['lanes'].astype(int)

detectors = detectors.drop(detectors[detectors['detid']=='N29121Z'].index)


utd_detectors = utd.merge(detectors, on='linkid', how='left')

In [101]:
utd.to_csv('utd.csv')
utd

,utd,utd_long,utd_lat,order,piece,linkid,group,utd_citycode
0,89398,-2.264742,53.485628,1,1,0,0.1,manchester
1,89399,-2.264694,53.486187,2,1,0,0.1,manchester
2,89400,-2.264653,53.486820,3,1,0,0.1,manchester
3,89401,-2.264374,53.487190,4,1,0,0.1,manchester
4,89402,-2.263860,53.487496,5,1,0,0.1,manchester
...,...,...,...,...,...,...,...,...
937,90335,-2.252004,53.481483,1,1,180,180.1,manchester
938,90336,-2.252378,53.481538,2,1,180,180.1,manchester
939,90337,-2.252817,53.481670,3,1,180,180.1,manchester
940,90338,-2.253200,53.481823,4,1,180,180.1,manchester


In [102]:
detectors

,detectors,detid,detectors_length,pos,fclass,road,limit,detectors_citycode,lanes,linkid,detectors_long,detectors_lat
0,15204,N06111V,0.119161,0.116664,trunk,Great Ancoats Street,48,manchester,2,72,-2.230164,53.483429
1,15205,N06112W,0.084072,0.076377,trunk,Great Ancoats Street,48,manchester,3,76,-2.228188,53.482206
2,15206,N06141A,0.417602,0.414577,trunk,Oldham Road,48,manchester,2,48,-2.224160,53.488127
3,15207,N06141C,0.114765,0.112883,trunk,Oldham Road,48,manchester,2,44,-2.231587,53.485518
4,15208,N06141Y,0.256407,0.249730,other,Thompson Street,0,manchester,1,42,-2.232718,53.487730
...,...,...,...,...,...,...,...,...,...,...,...,...
176,15380,N06482F,0.243023,0.229623,trunk,Chapel Street,32,manchester,2,127,-2.259198,53.483305
177,15381,N06482K,0.035744,0.020701,other,St Philip's Place,0,manchester,2,129,-2.262977,53.483618
178,15382,N06482J,0.107132,0.103730,trunk,Chapel Street,32,manchester,2,130,-2.264217,53.483308
179,15383,N06551T,0.446633,0.369536,trunk,Crescent,32,manchester,2,132,-2.265183,53.483210


In [103]:
detectors['linkid'].count()

180

In [104]:
print(detectors[detectors['linkid'].duplicated()])

Empty DataFrame
Columns: [detectors, detid, detectors_length, pos, fclass, road, limit, detectors_citycode, lanes, linkid, detectors_long, detectors_lat]
Index: []


In [90]:

utd_detectors.to_csv('utd_detectors.csv')
utd_detectors

,utd,utd_long,utd_lat,order,piece,linkid,group,utd_citycode,detectors,detid,length,pos,fclass,road,limit,detectors_citycode,lanes,detectors_long,detectors_lat
0,89398,-2.264742,53.485628,1,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
1,89399,-2.264694,53.486187,2,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
2,89400,-2.264653,53.486820,3,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
3,89401,-2.264374,53.487190,4,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
4,89402,-2.263860,53.487496,5,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,90335,-2.252004,53.481483,1,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
938,90336,-2.252378,53.481538,2,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
939,90337,-2.252817,53.481670,3,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
940,90338,-2.253200,53.481823,4,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538


In [117]:
num_utd = 0
'''build node.csv'''
print('converting utd data into gmns format...')


node_utd = pd.DataFrame()
node_utd['name'] = None
node_utd['x_coord'] = None
node_utd['y_coord'] = None
node_utd['z_coord'] = None
node_utd['node_type'] = None
node_utd['ctrl_type'] = None
node_utd['zone_id'] = None
node_utd['parent_node_id'] = None
node_utd['geometry'] = None

for i in range(len(utd)-1):
    if utd.loc[i+1,'order'] > utd.loc[i,'order']:
        node_utd = node_utd.append({'name': utd.loc[i,'utd'],\
                                    'x_coord': utd.loc[i,'utd_long'], \
                                    'y_coord': utd.loc[i,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i,'utd_long'].astype(str) + " " + utd.loc[i,'utd_lat'].astype(str) +")"}, ignore_index=True)
        num_utd += 1
    else:
        
        node_utd = node_utd.append({'name': utd.loc[i,'utd'],\
                                    'x_coord': utd.loc[i,'utd_long'], \
                                    'y_coord': utd.loc[i,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i,'utd_long'].astype(str) + " " + utd.loc[i,'utd_lat'].astype(str) +")"}, ignore_index=True)

node_utd = node_utd.append({'name': utd.loc[i+1,'utd'],\
                                    'x_coord': utd.loc[i+1,'utd_long'], \
                                    'y_coord': utd.loc[i+1,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i+1,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i+1,'utd_long'].astype(str) + " " + utd.loc[i+1,'utd_lat'].astype(str) +")"}, ignore_index=True)


node_utd.index.name = 'node_id'
node_utd.index += 100000001 #index from 0



node_utd.to_csv('node_utd.csv')
print('node_utd.csv generated!')

converting utd data into gmns format...
node_utd.csv generated!


In [118]:
num_utd

761

In [106]:
node_utd

,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,89398,-2.264742,53.485628,None,0,None,None,None,POINT (-2.264742 53.48562800000001)
100000002,89399,-2.264694,53.486187,None,0,None,None,None,POINT (-2.264694 53.486187)
100000003,89400,-2.264653,53.486820,None,0,None,None,None,POINT (-2.264653 53.48682)
100000004,89401,-2.264374,53.487190,None,0,None,None,None,POINT (-2.264374 53.48719000000001)
100000005,89402,-2.263860,53.487496,None,0,None,None,None,POINT (-2.26386 53.48749599999999)
...,...,...,...,...,...,...,...,...,...
100000938,90335,-2.252004,53.481483,None,180,None,None,None,POINT (-2.2520041 53.48148270000001)
100000939,90336,-2.252378,53.481538,None,180,None,None,None,POINT (-2.252378 53.481538)
100000940,90337,-2.252817,53.481670,None,180,None,None,None,POINT (-2.2528169 53.4816695)


In [107]:
utd_detectors

,utd,utd_long,utd_lat,order,piece,linkid,group,utd_citycode,detectors,detid,detectors_length,pos,fclass,road,limit,detectors_citycode,lanes,detectors_long,detectors_lat
0,89398,-2.264742,53.485628,1,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
1,89399,-2.264694,53.486187,2,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
2,89400,-2.264653,53.486820,3,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
3,89401,-2.264374,53.487190,4,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
4,89402,-2.263860,53.487496,5,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,90335,-2.252004,53.481483,1,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
938,90336,-2.252378,53.481538,2,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
939,90337,-2.252817,53.481670,3,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
940,90338,-2.253200,53.481823,4,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538


In [108]:
'''build link_utd.csv'''
link_utd = pd.DataFrame()
link_utd['name'] = None
link_utd['corridor_id'] = None
link_utd['corridor_link_order'] = None
link_utd['from_node_id'] = None
link_utd['to_node_id'] = None
link_utd['directed'] = None
link_utd['geometry_id'] = None
link_utd['geometry'] = None
link_utd['dir_flag'] = None
link_utd['parent_link_id'] = None
link_utd['length'] = None
link_utd['grade'] = None
link_utd['facility_type'] = None
link_utd['capacity'] = None
link_utd['free_speed'] = None
link_utd['lanes'] = None

for i in range(0,len(utd_detectors)-1):
    if utd.loc[i+1,'order'] > utd.loc[i,'order']:
        link_utd = link_utd.append({'name': utd_detectors.loc[i,'utd'],\
                                    'corridor_id': utd_detectors.loc[i,'road'],\
                                    'corridor_link_order' : utd_detectors.loc[i,'order'],\
                                    'from_node_id': node_utd[(node_utd['x_coord']==utd_detectors.loc[i,'utd_long']) & \
                                                    (node_utd['y_coord']==utd_detectors.loc[i,'utd_lat'])].index.values[0], \
                                    'to_node_id': node_utd[(node_utd['x_coord']==utd_detectors.loc[i+1,'utd_long']) & \
                                                    (node_utd['y_coord']==utd_detectors.loc[i+1,'utd_lat'])].index.values[0],\
                                    'directed': None,\
                                    'geometry_id': None,\
                                    'geometry': "LINESTRING (" + utd_detectors.loc[i,'utd_long'].astype(str) + " " + \
                                                utd_detectors.loc[i,'utd_lat'].astype(str) + "," + \
                                                utd_detectors.loc[i+1,'utd_long'].astype(str) +" "+ \
                                                utd_detectors.loc[i+1,'utd_lat'].astype(str) + ")",\
                                    'dir_flag': None,\
                                    'parent_link_id': None,\
                                    'length': None,\
                                    'grade': utd_detectors.loc[i,'fclass'],\
                                    'facility_type': None ,\
                                    'capacity':None,\
                                    'free_speed':utd_detectors.loc[i,'limit'],\
                                    'lanes': utd_detectors.loc[i,'lanes']}, ignore_index=True)
link_utd.index.name = 'link_id'
link_utd.index += 100000001


link_utd.to_csv('link_utd.csv')
print('link_utd.csv generated!')


link_utd.csv generated!


In [109]:
link_utd

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,,
100000001,89398,Silk Street,1,100000001,100000002,None,None,"LINESTRING (-2.264742 53.48562800000001,-2.264...",None,None,None,primary,None,None,48.0,1.0
100000002,89399,Silk Street,2,100000002,100000003,None,None,"LINESTRING (-2.264694 53.486187,-2.264653 53.4...",None,None,None,primary,None,None,48.0,1.0
100000003,89400,Silk Street,3,100000003,100000004,None,None,"LINESTRING (-2.264653 53.48682,-2.264374 53.48...",None,None,None,primary,None,None,48.0,1.0
100000004,89401,Silk Street,4,100000004,100000005,None,None,"LINESTRING (-2.264374 53.48719000000001,-2.263...",None,None,None,primary,None,None,48.0,1.0
100000005,89402,Silk Street,5,100000005,100000006,None,None,"LINESTRING (-2.26386 53.48749599999999,-2.2628...",None,None,None,primary,None,None,48.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000757,90333,Old Mill Street,2,100000936,100000937,None,None,"LINESTRING (-2.223232 53.48135500000001,-2.224...",None,None,None,tertiary,None,None,0.0,1.0
100000758,90335,Leftbank,1,100000938,100000939,None,None,"LINESTRING (-2.2520041 53.48148270000001,-2.25...",None,None,None,other,None,None,48.0,1.0
100000759,90336,Leftbank,2,100000939,100000940,None,None,"LINESTRING (-2.252378 53.481538,-2.2528169 53....",None,None,None,other,None,None,48.0,1.0


In [151]:
'''build measurement_utd.csv''' 

files= os.listdir(os.getcwd()) 
for file in files: 
    if file[:7] == 'Reading':
        reading = pd.read_csv(file)
        break

# reading = reading[pd.to_datetime(reading['measurement_tstamp'], format='%Y-%m-%d %H:%M:%S')<datetime.datetime.strptime('2015-04-01 02:00:00', '%Y-%m-%d %H:%M:%S')]
reading = reading.loc[0:1500]

measurement_tmc = pd.DataFrame()
measurement_tmc['name'] = None
measurement_tmc['corridor_id'] = None
measurement_tmc['corridor_link_order'] = None
measurement_tmc['from_node_id'] = None
measurement_tmc['to_node_id'] = None
measurement_tmc['timestamp'] = None
measurement_tmc['volume'] = None
measurement_tmc['travel_time'] = None
measurement_tmc['speed'] = None
measurement_tmc['reference_speed'] = None
measurement_tmc['density'] = None
measurement_tmc['queue'] = None
measurement_tmc['notes'] = None


gp = reading.groupby('measurement_tstamp')
for key, form in gp:
    # print(key)
    for i in link_tmc.index:
        form_selected = form[form['_vatmc_code']==link_tmc['name'][i]]
        if len(form_selected)>0:
        # break
            measurement_tmc = measurement_tmc.append({'name': link_tmc['name'][i],\
                                            'corridor_id': link_tmc['corridor_id'][i],\
                                            'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                            'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                            'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                            'timestamp': form_selected['measurement_tstamp'].values[0][0:10]+'T'+form_selected['measurement_tstamp'].values[0][11:13]+':'+form_selected['measurement_tstamp'].values[0][14:16],\
                                            'volume': None,\
                                            'travel_time': link_tmc['length'][i]/form_selected['speed'].values[0],\
                                            'speed': form_selected['speed'].values[0],\
                                            'reference_speed': form_selected['reference_speed'].values[0],\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)
        else:
            measurement_tmc = measurement_tmc.append({'name': link_tmc['name'][i],\
                                            'corridor_id': link_tmc['corridor_id'][i],\
                                            'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                            'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                            'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                            'timestamp': None,\
                                            'volume': None,\
                                            'travel_time': None,\
                                            'speed': None,\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)

measurement_tmc.to_csv('measurement_tmc.csv',index = False)
print('measurement_tmc.csv generated!')

measurement_tmc.csv generated!


In [152]:
measurement_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,timestamp,volume,travel_time,speed,reference_speed,density,queue,notes
0,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,2015-04-01T00:00,None,0.000877727,44,44,None,None,None
1,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,2015-04-01T00:00,None,0.00374761,44,44,None,None,None
2,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,2015-04-01T00:00,None,0.0104522,49,49,None,None,None
3,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,2015-04-01T00:00,None,0.0142206,48,48,None,None,None
4,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,2015-04-01T00:00,None,0.0061819,48,48,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,None,None,None,None,None,None,None,None
1608,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,None,None,None,None,None,None,None,None
1609,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,None,None,None,None,None,None,None,None
1610,110+05711,VA-7_WESTBOUND,48.0,100000062,100000063,None,None,None,None,None,None,None,None


## step 2 map matching

In [153]:
'''build trace.csv'''
'''trace_id is numeric'''
trace = pd.DataFrame()
trace['corridor_id'] = None
trace['agent_id'] = None
trace['date'] = None
trace['tmc'] = None
trace['trace_id'] = None
trace['hh'] = None
trace['mm'] = None
trace['ss'] = None
trace['y_coord'] = None
trace['x_coord'] = None

agent_id = 1
trace_id = 0

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        trace = trace.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'x_coord': tmc.loc[i,'start_longitude']}, ignore_index=True)
        trace_id +=1
    else:
        trace = trace.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'x_coord': tmc.loc[i,'start_longitude']}, ignore_index=True)
        trace_id += 1

        trace = trace.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'x_coord': tmc.loc[i,'end_longitude']}, ignore_index=True)
        agent_id += 1
        trace_id = 0
        
trace = trace.append({'corridor_id': tmc.loc[i+1,'road'] + '_' + tmc.loc[i+1,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i+1,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude']}, ignore_index=True)
trace_id +=1
trace = trace.append({'corridor_id': tmc.loc[i+1,'road'] + '_' + tmc.loc[i+1,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i+1,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'x_coord': tmc.loc[i+1,'end_longitude']}, ignore_index=True)

trace.to_csv('trace.csv')
print('trace.csv generated!')

trace.csv generated!


In [154]:
trace

,corridor_id,agent_id,date,tmc,trace_id,hh,mm,ss,y_coord,x_coord
0,VA-7_EASTBOUND,1,None,110N05711,0,None,None,None,39.0144,-77.3715
1,VA-7_EASTBOUND,1,None,110-05710,1,None,None,None,39.0141,-77.3709
2,VA-7_EASTBOUND,1,None,110N05710,2,None,None,None,39.0127,-77.3684
3,VA-7_EASTBOUND,1,None,110-05709,3,None,None,None,39.0086,-77.3605
4,VA-7_EASTBOUND,1,None,110-05708,4,None,None,None,39.0028,-77.3502
...,...,...,...,...,...,...,...,...,...,...
59,VA-7_WESTBOUND,2,None,110+05710,27,None,None,None,39.0030,-77.3500
60,VA-7_WESTBOUND,2,None,110P05710,28,None,None,None,39.0090,-77.3609
61,VA-7_WESTBOUND,2,None,110+05711,29,None,None,None,39.0125,-77.3673
62,VA-7_WESTBOUND,2,None,110P05711,30,None,None,None,39.0143,-77.3707


In [12]:
"Get the OSM Network"
network_type = 'auto'
import osm2gmns as og
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[-4:] == '.osm':
        net = og.getNetFromOSMFile(file,network_type=(network_type), default_lanes=True, default_speed=True)
        break

og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=os.getcwd())

osm2gmns, version 0.4.2


In [13]:
# """install MapMatching4GMNS"""
# !pip install MapMatching4GMNS --upgrade


if __name__ == "__main__":
    #If the output data exists, please remove.
    if(os.path.exists("agent.csv")):
       os.remove("agent.csv")
    if(os.path.exists("route.csv")):
       os.remove("route.csv")
    if(os.path.exists("link_performance.csv")):
       os.remove("link_performance.csv")   
    
    start = time.time()
    MapMatching4GMNS.map_match()
    end = time.time()
    
    print('MapMatching4GMNS time cost: %.6f seconds' % (end - start))
    print("The output data is generated!")

call MapMatching4GMNS  dynamic library
\MapMatching4GMNS run starts
\MapMatching4GMNS run completes
MapMatching4GMNS time cost: 0.651376 seconds
The output data is generated!


## step 3 link performance of osm

In [155]:
link_road = pd.read_csv('link.csv', low_memory=False)
trace = pd.read_csv('trace.csv', low_memory=False,index_col=None)
measurement_tmc = pd.read_csv('measurement_tmc.csv',low_memory=False,index_col=None)
mapping = pd.read_csv('route.csv', low_memory=False,index_col=None)


In [156]:
mapping

,agent_id,from_node_id,to_node_id,timestamp,cumu_distance,hhmmss,trace_id,travel_time,delay,geometry
0,1,14870,9642,0,212.49,0000:00,1,0,0,"LINESTRING (-77.37091289999999 39.0140956, -77..."
1,1,9642,9643,0,253.55,0000:00,2,0,0,"LINESTRING (-77.36887400000001 39.0130269, -77..."
2,1,9643,9644,0,661.67,0000:00,2,0,0,"LINESTRING (-77.368475 39.0128263, -77.3679666..."
3,1,9644,12392,0,1067.71,0000:00,2,0,0,"LINESTRING (-77.36452420000001 39.0108146, -77..."
4,1,12392,9992,0,1360.15,0000:00,3,0,0,"LINESTRING (-77.3606994 39.0086931, -77.360374..."
...,...,...,...,...,...,...,...,...,...,...
277,2,13667,2478,0,19159.35,0000:00,27,0,0,"LINESTRING (-77.3552028 39.0059016, -77.355703..."
278,2,2478,2479,0,19386.54,0000:00,28,0,0,"LINESTRING (-77.3557033 39.0061814, -77.356692..."
279,2,2479,12397,0,19722.92,0000:00,28,0,0,"LINESTRING (-77.3578304 39.0073823, -77.357980..."
280,2,12397,12828,0,20068.50,0000:00,28,0,0,"LINESTRING (-77.360985 39.009155, -77.3614358 ..."


In [15]:
measurement_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,timestamp,volume,travel_time,speed,reference_speed,density,queue,notes
0,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,2015-04-01T00:00,NaN,0.000878,44.0,44.0,NaN,NaN,NaN
1,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,2015-04-01T00:00,NaN,0.003748,44.0,44.0,NaN,NaN,NaN
2,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,2015-04-01T00:00,NaN,0.010452,49.0,49.0,NaN,NaN,NaN
3,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,2015-04-01T00:00,NaN,0.014221,48.0,48.0,NaN,NaN,NaN
4,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,2015-04-01T00:00,NaN,0.006182,48.0,48.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,2015-04-01T02:45,NaN,0.005421,47.0,47.0,NaN,NaN,NaN
2104,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,2015-04-01T02:45,NaN,0.014971,48.0,48.0,NaN,NaN,NaN
2105,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,2015-04-01T02:45,NaN,0.008612,49.0,49.0,NaN,NaN,NaN
2106,110+05711,VA-7_WESTBOUND,48.0,100000062,100000063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
trace

,Unnamed: 0,corridor_id,agent_id,date,tmc,trace_id,hh,mm,ss,y_coord,x_coord
0,0,VA-7_EASTBOUND,1,NaN,110N05711,0,NaN,None,NaN,39.0144,-77.3715
1,1,VA-7_EASTBOUND,1,NaN,110-05710,1,NaN,None,NaN,39.0141,-77.3709
2,2,VA-7_EASTBOUND,1,NaN,110N05710,2,NaN,None,NaN,39.0127,-77.3684
3,3,VA-7_EASTBOUND,1,NaN,110-05709,3,NaN,None,NaN,39.0086,-77.3605
4,4,VA-7_EASTBOUND,1,NaN,110-05708,4,NaN,None,NaN,39.0028,-77.3502
...,...,...,...,...,...,...,...,...,...,...,...
59,59,VA-7_WESTBOUND,2,NaN,110+05710,27,NaN,None,NaN,39.0030,-77.3500
60,60,VA-7_WESTBOUND,2,NaN,110P05710,28,NaN,None,NaN,39.0090,-77.3609
61,61,VA-7_WESTBOUND,2,NaN,110+05711,29,NaN,None,NaN,39.0125,-77.3673
62,62,VA-7_WESTBOUND,2,NaN,110P05711,30,NaN,None,NaN,39.0143,-77.3707


In [168]:
link_road

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway
0,NaN,0,5964128,4,5,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1878249 38.9744185, -77.187663...",auto,1
1,NaN,1,5964128,5,4,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1869767 38.9755412, -77.187013...",auto,1
2,NaN,2,5964129,6,7,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.194886 38.9751513, -77.1950799...",auto,1
3,NaN,3,5964129,7,6,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196016...",auto,1
4,NaN,4,5964129,7,8,1,85.661050,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196222...",auto,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30698,Sugarland Road,31152,943412232,3065,3723,1,74.086594,2,35,NaN,tertiary,5,"LINESTRING (-77.3798798 39.0078527, -77.379143...",auto,1
30699,Sugarland Road,31153,943412232,3723,3065,1,74.086594,2,35,NaN,tertiary,5,"LINESTRING (-77.37914309999999 39.0075118, -77...",auto,1
30700,Sugarland Road,31154,943412232,3723,7903,1,71.834767,2,35,NaN,tertiary,5,"LINESTRING (-77.37914309999999 39.0075118, -77...",auto,1
30701,Sugarland Road,31155,943412232,7903,3723,1,71.834767,2,35,NaN,tertiary,5,"LINESTRING (-77.37844440000001 39.0071617, -77...",auto,1


In [190]:
measurement_osm = pd.DataFrame()
measurement_osm_sub = pd.DataFrame()

measurement_osm['link_id'] = None
measurement_osm['from_node_id'] = None
measurement_osm['to_node_id'] = None
measurement_osm['lanes'] = None
measurement_osm['length'] = None
measurement_osm['time_period'] = None
measurement_osm['date'] = None
measurement_osm['geometry'] = None
measurement_osm['volume'] = None
measurement_osm['speed'] = None


link_road_dict = {}
gp = link_road.groupby(['from_node_id','to_node_id'])
for key, form in gp:
    link_road_dict[key] = {
        'link_id':form['link_id'].values[0],
        'from_node_id':form['from_node_id'].values[0],
        'to_node_id':form['to_node_id'].values[0],
        'lanes':form['lanes'].values[0],
        'length':form['length'].values[0],
        'geometry':form['geometry'].values[0],
        }


trace_dict = {}
gp = trace.groupby(['agent_id','trace_id'])
for key, form in gp:
    trace_dict[key] = {
        'tmc':form['tmc'].values[0]
        }


measurement_tmc_dict = {}
gp = measurement_tmc.groupby('name')
for key, form in gp:
    measurement_tmc_dict[key] = {
        'timestamp':form['timestamp'].tolist(),
        'volume':form['volume'].tolist(),
        'speed':form['speed'].tolist()
        }


for i in mapping.index:
    link_road_selected = link_road_dict[(mapping.loc[i,'from_node_id'],mapping.loc[i,'to_node_id'])] #find osm link from mapping 
    trace_selected = trace_dict[(mapping.loc[i,'agent_id'],mapping.loc[i,'trace_id'])] #find tmc link from mapping
    measurement_tmc_dict_selected = measurement_tmc_dict[trace_selected['tmc']] #find tmc performance from tmc link

    for j in range(0,len(measurement_tmc_dict_selected['timestamp']),3):
        try:
            measurement_osm_sub = measurement_osm_sub.append({'link_id': link_road_selected['link_id'],\
                                            'from_node_id': link_road_selected['from_node_id'],\
                                            'to_node_id': link_road_selected['to_node_id'],\
                                            'lanes': link_road_selected['lanes'],\
                                            'length': round(link_road_selected['length']),\
                                            'time_period': measurement_tmc_dict_selected['timestamp'][j][-5:-3] + \
                                                            measurement_tmc_dict_selected['timestamp'][j][-2:] + '_'+ \
                                                            measurement_tmc_dict_selected['timestamp'][j+3][-5:-3] + \
                                                            measurement_tmc_dict_selected['timestamp'][j+3][-2:],\
                                            'date': measurement_tmc_dict_selected['timestamp'][j][:10],\
                                            'geometry': link_road_selected['geometry'],\
                                            'volume': np.sum(measurement_tmc_dict_selected['volume'][j:j+3]),\
                                            'speed': round(np.mean(measurement_tmc_dict_selected['speed'][j:j+3]))}, ignore_index=True)

        except:
            break
            # measurement_osm_sub = measurement_osm_sub.append({'link_id': link_road_selected['link_id'],\
            #                                 'from_node_id': link_road_selected['from_node_id'],\
            #                                 'to_node_id': link_road_selected['to_node_id'],\
            #                                 'lanes': link_road_selected['lanes'],\
            #                                 'length': round(link_road_selected['length']),\
            #                                 'time_period': None,\
            #                                 'date': None,\
            #                                 'geometry': link_road_selected['geometry'],\
            #                                 'volume': None,\
            #                                 'speed': None}, ignore_index=True)


    measurement_osm = measurement_osm.append(measurement_osm_sub)
    measurement_osm_sub = pd.DataFrame()
measurement_osm.to_csv('measurement_osm.csv')
print("measurement_osm.csv is generated!")

measurement_osm.csv is generated!


In [191]:
measurement_osm

,link_id,from_node_id,to_node_id,lanes,length,time_period,date,geometry,volume,speed
0,18653.0,14870.0,9642.0,3.0,212.0,0000_0015,2015-04-01,"LINESTRING (-77.37091289999999 39.0140956, -77...",NaN,44.0
1,18653.0,14870.0,9642.0,3.0,212.0,0015_0030,2015-04-01,"LINESTRING (-77.37091289999999 39.0140956, -77...",NaN,44.0
2,18653.0,14870.0,9642.0,3.0,212.0,0030_0045,2015-04-01,"LINESTRING (-77.37091289999999 39.0140956, -77...",NaN,44.0
3,18653.0,14870.0,9642.0,3.0,212.0,0045_0100,2015-04-01,"LINESTRING (-77.37091289999999 39.0140956, -77...",NaN,44.0
4,18653.0,14870.0,9642.0,3.0,212.0,0100_0115,2015-04-01,"LINESTRING (-77.37091289999999 39.0140956, -77...",NaN,44.0
...,...,...,...,...,...,...,...,...,...,...
3,25491.0,12828.0,12388.0,3.0,399.0,0045_0100,2015-04-01,"LINESTRING (-77.3642456 39.0109544, -77.364827...",NaN,41.0
4,25491.0,12828.0,12388.0,3.0,399.0,0100_0115,2015-04-01,"LINESTRING (-77.3642456 39.0109544, -77.364827...",NaN,41.0
5,25491.0,12828.0,12388.0,3.0,399.0,0115_0130,2015-04-01,"LINESTRING (-77.3642456 39.0109544, -77.364827...",NaN,41.0
6,25491.0,12828.0,12388.0,3.0,399.0,0130_0145,2015-04-01,"LINESTRING (-77.3642456 39.0109544, -77.364827...",NaN,41.0


In [192]:
print( __name__)
print(os.path.dirname(os.path.realpath('__file__')))
print(os.path.realpath('__file__'))
print(os.getcwd())

__main__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/__file__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
